In [1]:
import os
import random
import math

import numpy as np
import haven.db as db 

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [6]:
TEST = True

table = 'movement_model_raw_features_m1_a2' if not TEST else 'movement_model_raw_features_m1_a2_test'
data = read_data_w_cache(
    f'select * from {table}'
)
data = data[data['_selected'].isin([True, False])]
data.head()

,_decision,_choice,_selected,_individual,normed_distance,origin_h3_index,next_h3_index,h3_index,time,_train,tag_key
0,1,1,True,172904,0.408506,8422d0bffffffff,8422d01ffffffff,8422d01ffffffff,2017-11-02,False,172904
1,1,2,False,172904,0.424598,8422d0bffffffff,8422d01ffffffff,8422d03ffffffff,2017-11-02,False,172904
2,1,3,False,172904,0.814645,8422d0bffffffff,8422d01ffffffff,8422d05ffffffff,2017-11-02,False,172904
3,1,4,False,172904,0.702011,8422d0bffffffff,8422d01ffffffff,8422d07ffffffff,2017-11-02,False,172904
4,1,5,False,172904,0.446883,8422d0bffffffff,8422d01ffffffff,8422d09ffffffff,2017-11-02,False,172904


In [7]:
df = data.merge(
    data[['_individual']].drop_duplicates().reset_index(drop=True).reset_index(),
    on='_individual'
)
del df['_individual']
df = df.rename({'index': '_individual'}, axis=1)
df.head()

,_decision,_choice,_selected,normed_distance,origin_h3_index,next_h3_index,h3_index,time,_train,tag_key,_individual
0,1,1,True,0.408506,8422d0bffffffff,8422d01ffffffff,8422d01ffffffff,2017-11-02,False,172904,0
1,1,2,False,0.424598,8422d0bffffffff,8422d01ffffffff,8422d03ffffffff,2017-11-02,False,172904,0
2,1,3,False,0.814645,8422d0bffffffff,8422d01ffffffff,8422d05ffffffff,2017-11-02,False,172904,0
3,1,4,False,0.702011,8422d0bffffffff,8422d01ffffffff,8422d07ffffffff,2017-11-02,False,172904,0
4,1,5,False,0.446883,8422d0bffffffff,8422d01ffffffff,8422d09ffffffff,2017-11-02,False,172904,0


In [8]:
df.groupby('_train').size()

_train
False    27178
dtype: int64

In [9]:
table = 'movement_model_features_m1_a2' if not TEST else 'movement_model_features_m1_a2_test'

db.write_data(
    df, table, ['tag_key']
)